This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import collections
import json 
import operator
import numpy as np

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=' + API_KEY
r = requests.get(url)
#print(r.json())


# Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY
r = requests.get(url)
r_json = r.json()
#r_json

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# Convert the returned JSON object into a Python dictionary

In [18]:
r_json.keys()

dict_keys(['dataset'])

In [19]:
r_json['dataset'].keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [28]:
# Inspect column_name information
r_json['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [21]:
# Data is organized as a list
r_json['dataset']['data'][0:2]

[['2019-04-10',
  None,
  75.1,
  71.25,
  72.35,
  None,
  275232.0,
  19935924.0,
  None,
  None,
  None],
 ['2019-04-09',
  None,
  75.8,
  73.3,
  75.3,
  None,
  183859.0,
  13789864.0,
  None,
  None,
  None]]

# Calculate what the highest and lowest opening prices were for the stock in this period.

In [30]:
# Investigating the column names list and its indexes, in order to match with the 'data' lists.

for i, column in enumerate(r_json['dataset']['column_names']):
    print('{} : {}'.format(i, column))

0 : Date
1 : Open
2 : High
3 : Low
4 : Close
5 : Change
6 : Traded Volume
7 : Turnover
8 : Last Price of the Day
9 : Daily Traded Units
10 : Daily Turnover


In [31]:
# Investigating the data lists, and see how it matches to 'column names'

for i, data in enumerate(r_json['dataset']['data'][0]):
    print('{} : {}'.format(i, data))

0 : 2017-12-29
1 : 51.76
2 : 51.94
3 : 51.45
4 : 51.76
5 : None
6 : 34640.0
7 : 1792304.0
8 : None
9 : None
10 : None


In [7]:
# The highest 'High' in 2017.

l_high = []
for data in r_json['dataset']['data']:
    l_high.append(data[2])

#print('{}'.format(high))   
max(l_high)

53.54

In [8]:
# The lowest 'Low' in 2017

l_low = []
for data in r_json['dataset']['data']:
    l_low.append(data[3])

min(l_low)

33.62

# What was the largest change in any one day (based on High and Low price)?


In [39]:
# In the data list store difference between 'High' & 'Low' values
one_day_change = []
for data in r_json['dataset']['data']:    
    one_day_change.append(data[2] - data[3])

max(one_day_change)

2.8100000000000023

# What was the largest change between any two days (based on Closing Price)?


In [9]:
# To find the largest closing price 'Close' between any two days,  created a list with all the closing prices.
# Secondly, with zip, created a list with the closing prices of two consecutive days. Lastly, with a 
# list comprehension subtracted each list and then got the maximum value, which give us 
# the largest change between any two days.

l_closing_price = []
for data in r_json['dataset']['data']:    
    l_closing_price.append(data[4])

#print('Day1:',closing_price[:-1])
#print('Day2:',closing_price[1:])

consec_days = zip(l_closing_price[:-1], l_closing_price[1:])
max([x[1] - x[0] for x in consec_days])

2.559999999999995

# What was the average daily trading volume during this year?

In [10]:
# To find the average value, appended all the 'Volume' numbers to a list, summed all the numbers in the list and 
# divided by 2, giving us the volume average.

l_vol = []
for data in r_json['dataset']['data']:    
    l_vol.append(data[6])
    
mean_vol = sum(l_vol)/len(l_vol)
mean_vol

89124.33725490196

# (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)